*Copyright (C) 2022 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Motion tracking


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.compiler.compiler import Compiler
from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.process.message_interface_enum import ActorType
from lava.magma.runtime.runtime import Runtime

from lava.magma.core.run_conditions import RunSteps

from lava.proc.lif.process import LIF
from sparse import Sparse
from live_plot import LivePlot, PyTutorialLiveMatrixReaderPMBokeh, PyTutorialLiveMatrixReaderPMMatplotLib2

from lava.lib.dnf.connect.connect import _configure_ops, \
    _compute_weights
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution

from dvs.file.dvs_file_input.process import DVSFileInput, PyDVSFileInputPMDense
from miscellaneous.c_injector.process import CInjector, CInjectorPMVecDense
from miscellaneous.c_spike_reader.process import CSpikeReader, \
    CSpikeReaderPMVecDense
from miscellaneous.py_spike_reader.process import PySpikeReader, PySpikeReaderPMDense
from bokeh.io import output_notebook

output_notebook()

# Process Params
true_height = 180
true_width = 240
file_path = "twoHands-2022_07_19_11_16_19.aedat4"
file_path = "run_7.aedat4"
flatten = True
down_sample_factor = 8
down_sample_mode = "max_pooling"  # max_pooling, down_sampling, convolution

down_sampled_shape = (true_height // down_sample_factor,
                      true_width // down_sample_factor)

num_neurons = (true_height // down_sample_factor) * \
              (true_width // down_sample_factor)
down_sampled_flat_shape = (num_neurons,)

# Define Processes
dvs_file_input = DVSFileInput(true_height=true_height,
                              true_width=true_width,
                              file_path=file_path,
                              flatten=flatten,
                              down_sample_factor=down_sample_factor,
                              down_sample_mode=down_sample_mode)
# replace with PyToNxAdapter?
c_injector = CInjector(shape=down_sampled_flat_shape)
sparse = Sparse(weights=np.eye(num_neurons) * 10)
lif = LIF(shape=down_sampled_shape, du=2000, dv=2000, vth=30)
c_spike_reader = CSpikeReader(shape=down_sampled_shape)
live_plot = LivePlot(shape=down_sampled_shape)

# Connect Processes
dvs_file_input.event_frame_out.connect(c_injector.in_port)
c_injector.out_port.connect(sparse.s_in)
sparse.a_out.reshape(new_shape=down_sampled_shape).connect(lif.a_in)
lif.s_out.connect(c_spike_reader.in_port)
c_spike_reader.out_port.connect(live_plot.frame_port)

# DNF connection
kernel = MultiPeakKernel(amp_exc=7,
                         width_exc=[7, 7],
                         amp_inh=-5,
                         width_inh=[14, 14])
#ops = [Convolution(kernel)]
#_configure_ops(ops, lif.s_out.shape, lif.a_in.shape)
#weights = _compute_weights(ops)
#connections = Sparse(weights=weights)
#con_ip = connections.s_in
#lif.s_out.reshape(new_shape=con_ip.shape).connect(con_ip)
#con_op = connections.a_out
#con_op.reshape(new_shape=lif.a_in.shape).connect(lif.a_in)

print(100 * "=")

# Running for num_steps time steps
print("Running : BEGIN")
exception_pm_map = {
DVSFileInput: PyDVSFileInputPMDense,
CInjector: CInjectorPMVecDense,
CSpikeReader: CSpikeReaderPMVecDense,
PySpikeReader: PySpikeReaderPMDense,
LivePlot: PyTutorialLiveMatrixReaderPMMatplotLib2
}
run_cfg = Loihi2HwCfg(exception_proc_model_map=exception_pm_map)
run_cnd = RunSteps(num_steps=600)

dvs_file_input.run(condition=run_cnd, run_cfg=run_cfg)
print("Running : END")

print(100 * "=")

print("Stopping : BEGIN")
dvs_file_input.stop()
print("Stopping : END")
